## Monte carlo test for proportion of co-occurring over non-co-occurring events 

In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# opening the input files with heatwave severity data 
thw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-coastal-sev.nc')
mhw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-ocean-sev-2.nc')

In [3]:
# using severity>1 to redefine data array as heatwave(1) and non-heatwave(0) days  
thw_da = xr.where(thw > 1.0, 1, 0)
mhw_da = xr.where(mhw > 1.0, 1, 0)

In [4]:
# First we compute the series of random numbers 
# Use Nrepeats =100 to start with adn then change to 10000 to compute the correct results
Ntime = thw_da.sizes["time"]
Nrepeats=100
r_arr = np.random.randint(Ntime-1,size=Nrepeats)

# New method:
Works straight for 1 point or all the points

In [5]:
# Create an array to store the proportions of events 
proportion_rand = xr.DataArray(np.ndarray([Nrepeats,thw_da.sizes['latitude'],thw_da.sizes['longitude']]),
                               dims=("Repeats","latitude","longitude"),
                               coords={"latitude":thw_da["latitude"],
                                       "longitude":thw_da["longitude"],
                                       "Repeats":np.arange(Nrepeats)
                                      }
                              )
proportion_rand

<xarray.DataArray (Repeats: 100, latitude: 137, longitude: 165)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * latitude   (latitude) float64 -10.0 -10.25 -10.5 ... -43.5 -43.75 -44.0
  * longitude  (longitude) float64 113.0 113.2 113.5 113.8 ... 153.5 153.8 154.0
  * Repeats    (Repeats) int64 0 1 2 3 4 5 6 7 8 ... 91 92 93 94 95 96 97 98 99

In [6]:
#Storing proportion values for each 'simulation'

for i,r in enumerate(r_arr):
    thw_ = thw_da.roll(time=r,roll_coords=False)
    co_occ = thw_.where(np.logical_and(thw_==1,mhw_da==1)).count(dim="time")
    non_co_occ = thw_.where(thw_==1).count(dim="time")
    proportion_rand.loc[{"Repeats":i}] = co_occ/non_co_occ

TypeError: float() argument must be a string or a number, not 'method'